In [31]:
pip install langchain_ollama

  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
Using cached httpcore-1.0.5-py3-none-any.whl (77 kB)
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.16.3
    Uninstalling httpcore-0.16.3:
      Successfully uninstalled httpcore-0.16.3
  Attempting uninstall: httpx
    Found existing installation: httpx 0.23.3
    Uninstalling httpx-0.23.3:
      Successfully uninstalled httpx-0.23.3
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dagshub 0.3.27 requires httpx~=0.23.0, but you have httpx 0.27.0 which is incompatible.
dagshub 0.3.27 requires rich~=13.1.0, but you have rich 13.7.1 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())


True

In [17]:
FIRE_API_KEY = os.getenv("FIRE_API_KEY")

In [13]:
from typing import List
from langchain_community.document_loaders import FireCrawlLoader
from document import Document

In [14]:
class DocumentLoader:
    def __init__(self, api_key: str):
        self.api_key = api_key

    def get_docs(self, url: str) -> List[Document]:
        """
        Retrieves documents from the specified URL using the FireCrawlLoader.

        Args:
            url (str): The URL to crawl for documents.

        Returns:
            List[Document]: A list of Document objects containing the retrieved content.
        """
        loader = FireCrawlLoader(
            api_key=self.api_key, url=url, mode="crawl"
        )

        raw_docs = loader.load()
        docs = [Document(page_content=doc.page_content, metadata=doc.metadata) for doc in raw_docs]

        return docs

In [18]:
doc = DocumentLoader(FIRE_API_KEY)

In [21]:
data= doc.get_docs("https://en.wikipedia.org/wiki/Python_(programming_language)")

In [29]:
data[0].page_content

'[Jump to content](#bodyContent)\n\n Main menu\n\nMain menu\n\nmove to sidebar hide\n\nNavigation\n\n*   [Main page](/wiki/Main_Page "Visit the main page [alt-shift-z]")\n    \n*   [Contents](/wiki/Wikipedia:Contents "Guides to browsing Wikipedia")\n    \n*   [Current events](/wiki/Portal:Current_events "Articles related to current events")\n    \n*   [Random article](/wiki/Special:Random "Visit a randomly selected article [alt-shift-x]")\n    \n*   [About Wikipedia](/wiki/Wikipedia:About "Learn about Wikipedia and how it works")\n    \n*   [Contact us](//en.wikipedia.org/wiki/Wikipedia:Contact_us "How to contact Wikipedia")\n    \n*   [Donate](https://donate.wikimedia.org/wiki/Special:FundraiserRedirector?utm_source=donate&utm_medium=sidebar&utm_campaign=C13_en.wikipedia.org&uselang=en "Support us by donating to the Wikimedia Foundation")\n    \n\nContribute\n\n*   [Help](/wiki/Help:Contents "Guidance on how to use and edit Wikipedia")\n    \n*   [Learn to edit](/wiki/Help:Introductio

In [33]:
from typing import List, Optional
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

def create_vector_store(docs, store_path: Optional[str] = None) -> FAISS:
    """
    Creates a FAISS vector store from a list of documents.

    Args:
        docs (List[Document]): A list of Document objects containing the content to be stored.
        store_path (Optional[str]): The path to store the vector store locally. If None, the vector store will not be stored.

    Returns:
        FAISS: The FAISS vector store containing the documents.
    """
    # Creating text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
    )

    texts = text_splitter.split_documents(docs)

    # Embedding object
    embeddings = OllamaEmbeddings(model="nomic-embed-text")

    # Create the FAISS vector store
    store = FAISS.from_documents(texts, embeddings)

    # Save the vector store locally if a path is provided
    if store_path:
        store.save_local(store_path)

    return store




In [34]:
# create vector store
saved_docs = "file"
store = create_vector_store(data,saved_docs)

# creating retriever
retriever = store.as_retriever()

ResponseError: failed to generate embedding

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def create_generate_chain(llm):
    """
    Creates a generate chain for answering code-related questions.

    Args:
        llm (LLM): The language model to use for generating responses.

    Returns:
        A callable function that takes a context and a question as input and returns a string response.
    """
    generate_template = """
    You are a helpful code assistant named Speckly. The user provides you with a code-related question whose content is represented by the following context parts (delimited by <context></context>).
    Use these to answer the question at the end.
    The files deal with Speckle Developer Documentation. You can assume that the user is either a civil engineer, architect, or a software developer.
    If you don't know the answer, just say that you don't know. Do NOT try to make up an answer.
    If the question is not related to the context, politely respond that you only answer questions related to the context.
    Provide as detailed an answer as possible and generate the code in Python (default) unless specifically mentioned by the user in the question.

    <context>
    {context}
    </context>

    <question>
    {input}
    </question>
    """

    generate_prompt = PromptTemplate(template=generate_template, input_variables=["context", "input"])

    # Create the generate chain
    generate_chain = generate_prompt | llm | StrOutputParser()

    return generate_chain


In [ ]:
# Create the generate chain
generate_chain = create_generate_chain(llm)